In [2]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing

In [3]:
housing = fetch_california_housing()  # Fetch your housing data
data = pd.DataFrame(
    housing.data, columns=housing.feature_names
)  # Load the fetched data into pandas dataframe
data["Price"] = housing.target  # Add target variable to dataframe
data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [4]:
from urllib.parse import urlparse
from mlflow.models import infer_signature

In [5]:
X = data.drop(columns=["Price"])
y = data["Price"]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
signature = infer_signature(X_train, y_train)

In [7]:
param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [5, 10, None],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2],
}

In [8]:
def hyperparameter_tuning(X_train, y_train, param_grid):
    rf = RandomForestRegressor()
    grid_search = GridSearchCV(
        estimator=rf,  # model to be tuned
        param_grid=param_grid,  # hyperparameter values to try
        cv=3,  # number of cross-validation folds
        n_jobs=-1,  # use all CPU cores for parallel processing
        verbose=1,  # display training progress
        scoring="neg_mean_squared_error",  # metric used to evaluate model performance
    )
    grid_search.fit(X_train, y_train)
    return grid_search

In [9]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000/")
mlflow.set_experiment("House Price Prediction With Hyperparameter Tuning")

2025/12/28 11:53:59 INFO mlflow.tracking.fluent: Experiment with name 'House Price Prediction With Hyperparameter Tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1766902139156, experiment_id='1', last_update_time=1766902139156, lifecycle_stage='active', name='House Price Prediction With Hyperparameter Tuning', tags={}>

In [10]:
with mlflow.start_run(run_name="RandomForest_Hyperparam_Tuning"):
    ## Perform Hyperparameter Tuning
    grid_search = hyperparameter_tuning(X_train, y_train, param_grid)
    ## Get the best model
    best_model = grid_search.best_estimator_
    ## Evaluate the best model
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    ## Log the best model
    mlflow.log_param("best_n_estimators", grid_search.best_params_["n_estimators"])
    mlflow.log_param("best_max_depth", grid_search.best_params_["max_depth"])
    mlflow.log_param(
        "best_min_samples_split", grid_search.best_params_["min_samples_split"]
    )
    mlflow.log_param(
        "best_min_samples_leaf", grid_search.best_params_["min_samples_leaf"]
    )
    mlflow.log_metric("mse", mse)

    ## Check if the tracking uri has been set or not
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
    if tracking_url_type_store != "file":
        mlflow.sklearn.log_model(
            best_model, name="model", registered_model_name="Best Random Forest Model"
        )
    else:
        mlflow.sklearn.log_model(best_model, name="model", signature=signature)

    print("Best Model Parameters:", grid_search.best_params_)
    print("Best Model MSE:", mse)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


Successfully registered model 'Best Random Forest Model'.
2025/12/28 11:56:47 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Best Random Forest Model, version 1
Created version '1' of model 'Best Random Forest Model'.


Best Model Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
Best Model MSE: 0.26071297621373707
🏃 View run RandomForest_Hyperparam_Tuning at: http://127.0.0.1:5000/#/experiments/1/runs/917296cf0e30424099f3e1eb7b7be537
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
